In [2]:
import numpy as np
import pandas as pd # Talvez fosse mais eficiente trabalhar com o Polars, porém não consigo instalá-lo na máquina do STF...
import nltk # Pacote para remoção de stopwords em português
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import unidecode 
import os
import re
import matplotlib.pyplot as plt
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'

# No caso, já preparei este conjunto de treino e teste acima anteriormente no R. Caso queira utilizar um conjunto qualquer, utilizar o bloco abaixo.

## Note que é necessário fazer algumas alterações no bloco abaixo para ele funcionar, como o diretório e nome do arquivo

### Além disso, o arquivo deve conter apenas duas colunas: Uma chamada 'texto' contendo as strings a serem analisadas, e outra com a classificação.

In [12]:
dataset = pd.read_csv('dados.csv')
corpus = []
for i in range(0,dataset.shape[0]): # Este index irá funcionar somente se a coluna texto for a primeira (índice 0). Deve-se alterar, c.c.
    texto = re.sub('[^a-zA-Z]',' ',unidecode.unidecode(dataset['texto'][i]) )
    texto = texto.lower()
    texto = texto.split() #convert into list
    ps = PorterStemmer()
    texto = [ps.stem(word) for word in texto if not word in set(stopwords.words('portuguese'))]
    texto = ' '.join(texto)
    corpus.append(texto)

['sr president',
 'marco rogerio',
 'obrigado',
 'sao muito rogerio sr president primeiro queria cumprimentar v exa decisao',
 'sr president primeiro quero cumprimentar v exa decisao madura equilibrada permit ambient saudavel curso investigacao ambito desta cpi ponderacao faco v exa relacao acordo procedimento concernent pedido quebra sigilo quanto requisicao documento ha acordo absolutament razoavel aqui relacao quebra faria ponderacao nao apreciassemo ness metodo devida avaliacao porqu medida invasiva ai voce la requisito quebra nao possivel fazer verificacao todo pedido constant pauta dia hoje nao gent aprovar quebra sigilo varejo preciso verificar fundamentacao razoabilidad pedido porqu deve imprescindivel investigacao entao ponderacao faco v exa nao inclua ness rol entendimento questao quebra sigilo',
 'nao veja president apena concordar part v exa certo',
 'certo intencao nao diss todo concordo relatora relacao ja ouvimo ai ha divergencia pontual aqui apena relacao ai ponto vista

In [13]:
# BoW
cv=CountVectorizer(max_features = 5000)
X=cv.fit_transform(corpus).toarray()
y=dataset.iloc[:,1].values # Coluna do classificador

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [20]:
# Fitting classifier to the Training set
classifier=RandomForestClassifier(n_estimators =400,criterion="entropy",random_state =0)
classifier.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=400, random_state=0)

Montando a função:

In [27]:
def modelar(X_train,y_train):

    # Random forest
    classifier=RandomForestClassifier(n_estimators =400,criterion="entropy",random_state =0)
    classifier.fit(X_train,y_train)

    # XGBoost:
    

    return classifier

In [29]:
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
bst.fit(X_train, y_train)


NameError: name 'XGBClassifier' is not defined

In [28]:
modelar(X_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=400, random_state=0)

In [21]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [23]:
# Making the Confusion Matrix
cm2 = confusion_matrix(y_test, y_pred)
cm2

array([[495, 139],
       [209, 332]], dtype=int64)